# Find Meme

## 0. preprocessing

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

CHAT_MODEL="deepseek-v3"
os.environ["OPENAI_API_KEY"]=os.environ.get("INFINITE_API_KEY") 
os.environ["OPENAI_BASE_URL"]=os.environ.get("INFINITE_BASE_URL")

In [2]:
from langchain_openai import OpenAIEmbeddings
baai_embedding = OpenAIEmbeddings(
    model="BAAI/bge-m3",
    base_url=os.environ.get("SF_BASE_URL"),
    api_key=os.environ.get("SF_API_KEY"),
)

In [ ]:
# test

baai_embedding.embed_query("I am Alice.") # test the embedding

## 1. embed data

In [4]:
# chromaDB setting

from langchain_chroma import Chroma

picture_paths = "../src/output/vv"
chroma_dir = "/scratch1/chroma_db"
docsearch_chroma = Chroma(
    embedding_function=baai_embedding,
    persist_directory=chroma_dir,
    collection_name="memes",
)


In [5]:
from langchain_core.documents import Document
titles = []
paths = []
documents = []
cnt = 0

for filepath, dirnames, filenames in os.walk(picture_paths):
    for filename in filenames:
        titles.append(filename)
        paths.append(filepath)
        doc = Document(page_content=filename, metadata={'path': filepath, 'id': str(cnt)})
        documents.append(doc)
        cnt += 1

docsearch_chroma.reset_collection()
for i in range(0,len(documents),64):
    docsearch_chroma.add_documents(documents[i:i+64])


In [ ]:
query = "如何评价原神"
docs = docsearch_chroma.similarity_search(query, k=6)
from PIL import Image


    
for result in docs:
    print(result)
    cur_path = os.path.join(result.metadata['path'], result.page_content)
    with Image.open(cur_path) as img:
        img.show()